In [202]:
# With numerical speed and computing prowess 
# The number package in python
# Domain Expansion:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import imageio

In [203]:
# Raw data from the internet
csv = [
    '003020600',
    '900305001',
    '001806400',
    '008102900',
    '700000008',
    '006708200',
    '002609500',
    '800203009',
    '005010300']


In [204]:
# Reformatting the data
data = []
for i in csv:
    list1 = []
    for char in i:
        list1.append(int(char))                         # This essentially turns this into
    data.append(list1)                                  # a 2D Array. I prefer to keep things that way


# For Visualization as .png. 
# Taken from GPT using simple imshow and label
# Many ways to do this, no need to overthink
# Just for Aesthetics
def vis(data):
    fig , ax = plt.subplots()
    cax = ax.imshow(
        data, 
        cmap='binary', 
        interpolation='nearest',
        vmin = 0,
        vmax = 20
        )
    for i in range(len(data)):
        for j in range(len(data)):
            ax.text(j, 
                    i, 
                    data[i][j], 
                    ha='center', 
                    va='center', 
                    color='black', 
                    fontsize=12
                    )

    ax.tick_params(
        top=False, 
        bottom=False, 
        left=False, 
        right=False, 
        labelleft=False, 
        labelbottom=False
        )
    
    plt.savefig(filename)
    plt.close()

vis(data)

# This one is for turning the .png into a .gif later
# Many ways to do this, again, just for aesthetics
frames = []
for i in range(0,5):
    filename = f'frame{i}.png'
    vis(data)
    frames.append(imageio.imread(filename))

C:\Users\rbnar\AppData\Local\Temp\ipykernel_12868\3053287447.py:53: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(filename))


Some Translators

In [205]:
def box(y,x):

    if (y%3) - 1 != 0: y_center = y - ((y%3)-1)                 # y%3 because each big block is 3 blocks high. I want to "quantize/group" it
    else: y_center = y                                        # I want to know the distance between the current y from the center
                                                                # This way. I can add/subtract back the distance to the current y
                                                                # to get y_center.

    if (x%3) - 1 != 0: x_center = x - ((x%3)-1)
    else: x_center = x

    # Kernel Density (Not "Kernel" from comsci but "Kernel" from Math)
    boxlist = [
        data[y_center-1][x_center-1] , data[y_center-1][x_center] , data[y_center-1][x_center+1] ,
        data[y_center][x_center-1] , data[y_center][x_center] , data[y_center][x_center+1] ,
        data[y_center+1][x_center-1] , data[y_center+1][x_center] , data[y_center+1][x_center+1]
    ]
    return boxlist


def col(y,x):
    collist = []
    for y in data:
        collist.append(y[x])
    return collist


def row(y,x):
    return data[y]

# my coordinate system is (y,x) instead of (x,y)
# and (0,0) is the top left cell of the puzzle
# (8,8) is the bottom right cell of the puzzle
print(f'Box Boundary: {box(4,3)}')
print(f'Column Boundary: {col(4,3)}')
print(f'Row Boundary: {row(4,3)}')

Box Boundary: [1, 0, 2, 0, 0, 0, 7, 0, 8]
Column Boundary: [0, 3, 8, 1, 0, 7, 6, 2, 0]
Row Boundary: [7, 0, 0, 0, 0, 0, 0, 0, 8]


Some Algorithms

In [206]:
# Generate list of blanks in the puzzle
# Asked GPT for this one for a better time complexity that doing double for loop
empty = [(y_0,x_0) for y_0 in range(len(data)) for x_0 in range(len(data[y_0])) if data[y_0][x_0] == 0]
print(len(empty))
#for y,x in empty: print(y,x)
basis = [1,2,3,4,5,6,7,8,9] 

49


In [207]:
# Cross Method
# Check Row, Column, and Box for Missing Number
def cross(y,x):
    cross = box(y,x) + col(y,x) + row(y,x)                      # Gives me all the numbers in the box, col, and row 

    cross = [ _ for _ in cross if _ != 0 ]                      # Removes all empty cells (zeroes) in the list "cross"
    subtract = [ _ for _ in basis if _ not in cross]            # checks what numbers arent in the box, col, and row


    return subtract


Computation

In [208]:
vis(data)
sweep = 5
filename = f'frame_{sweep}.png'

while len(empty) > 0:
    for y,x in empty:
        if len(cross(y,x)) == 1:
            data[y][x] = cross(y,x)[0]
            empty.remove((y,x))

            vis(data)
            sweep += 1

            frames.append(imageio.imread(filename))

C:\Users\rbnar\AppData\Local\Temp\ipykernel_12868\1622397658.py:14: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  frames.append(imageio.imread(filename))


Video

In [209]:
imageio.mimsave('output.gif', frames, duration=0.1)